In [ ]:
import qiskit
import ansatz
%load_ext autoreload
%autoreload 2

from custom_lib import quantum_circuit_ctx_generator


# Benchmark from quanvolutional1 to quanvolutional19

# for num_qubits in range(3, 10):
#     for i in range(1, 20):
#         function_name = f'quanvolutional{i}'
#         qc = qiskit.QuantumCircuit(num_qubits) 
#         qc = getattr(ansatz, function_name)(qc)

function_name = f'quanvolutional{3}'
qc = qiskit.QuantumCircuit(3) # Number of qubits
qc = getattr(ansatz, function_name)(qc)
qc.draw(output='mpl')

# qc = qiskit.transpile(qc, basis_gates=['h', 's', 'cx', 'rx', 'ry', 'rz'], optimization_level=3)
# qc.draw(output='mpl')

In [ ]:
# convert to readable files

from qimax import converter, constant

texts = []
texts.append(len(qc.data))
for gate in qc.data:
    name = gate.name.upper()
    params = None
    wires = converter.get_wires_of_gate(gate)
    if name == 'CX':
        type = 2
    else:
        type = 1
        gate_entries = constant.constant_gate[name]
        if name in ['RX', 'RY', 'RZ']:
            params = gate.params[0]
            gate_entries = gate_entries(params)
    texts.append(type)
    if type == 2:
        texts.append("{" + str(wires[0]) + ", "+ str(wires[1]) + "}")
    else:
        texts.append(wires[0])
        texts.append(gate_entries[0][0])
        texts.append(gate_entries[0][1])
        texts.append(gate_entries[1][0])
        texts.append(gate_entries[1][1])

with open('output.txt', 'w') as f:
    for item in texts:
        f.write("%s\n" % item)

In [ ]:
#  RESULT FROM QISKIT
qc_qiskit = (qiskit.quantum_info.Statevector.from_instruction(qc).data)
print(qc_qiskit)

In [ ]:
# # Calculate error

import numpy as np

def fidelity(state1: np.ndarray, state2: np.ndarray):
    state1 = np.expand_dims(state1, axis=0)
    state2 = np.expand_dims(state2, axis=0)
    return (np.abs(np.inner(np.conjugate(state1), state2))**2)[0][0]


def mse(state1: np.ndarray, state2: np.ndarray):
    return np.mean(np.abs(state1 - state2)**2)

qc_fpga = np.array([0,1,2,3,4,5,6,7])
print(fidelity(qc_qiskit, qc_fpga))
print(mse(qc_qiskit, qc_fpga))